In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM,Dense
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

In [ ]:
multi_data = pd.read_csv("data/flotation-cell.csv",parse_dates=True,index_col=0)

In [ ]:
def create_lags(df):
    for col in df.columns:
        df[col+"_1"] = df[col].shift(1)
    return df.dropna()

In [ ]:
# Look at the dataset because something is wrong ?

In [ ]:
multi_data.reset_index(inplace=True)
del multi_data['Date and time']

In [ ]:
multi_data.dropna(inplace=True)

In [ ]:
reframed_multi_data = create_lags(multi_data)

In [ ]:
reframed_multi_data.columns

In [ ]:
y = reframed_multi_data['Air flow rate']

In [ ]:
x = reframed_multi_data[['Upstream pH', 'CuSO4 added', 'Pulp level',
       'Feed rate', 'Feed rate_1', 'Upstream pH_1', 'CuSO4 added_1',
       'Pulp level_1', 'Air flow rate_1']]

In [ ]:
x_train = x[:2500]
x_test = x[2500:]

In [ ]:
y_train = y[:2500]
y_test = y[2500:]

In [ ]:
# Something fishy here
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)
y_train = scaler.fit_transform(y_train.values.reshape(-1,1))
y_test = scaler.fit_transform(y_test.values.reshape(-1,1))

x_train = x_train.reshape(x_train.shape[0],1,x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0],1,x_test.shape[1])

In [ ]:
x_train.shape

In [ ]:
# does hyper parameter tuning help ?
multi_lstm_model = Sequential()
multi_lstm_model.add(LSTM(100,input_shape=(x_train.shape[1],x_train.shape[2])))
multi_lstm_model.add(Dense(1))
multi_lstm_model.compile(loss="mse",optimizer="adam")

In [ ]:
multi_lstm_model.fit(x_train,y_train,nb_epoch=25,batch_size=100)

In [ ]:
y_pred = multi_lstm_model.predict(x_test)

In [ ]:
def plot_predictions(preds,y_test):
    plt.figure(figsize=(10, 5.5))
    plt.plot(preds,linestyle='-',color='b')
    plt.plot(y_test,linestyle='-',color='r')
    plt.legend(['Predicted','Actual'], loc=2)
    plt.title('Actual vs Predicted')

In [ ]:
plot_predictions(y_pred[:100],y_test[:100])

In [ ]:
r2_score(y_test,y_pred)